统计包含多少element

In [4]:
import json

# 读取 JSON 文件并解析为 Python 对象
def count_elements_in_json(json_file):
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return len(data)

# 指定 JSON 文件路径
json_file = "/home/qiang/projects/Digital_mental_health/Dataset/Final_dataset/IAS2EAS/IAS2EAS.json"

# 统计元素数量
element_count = count_elements_in_json(json_file)
print(f"JSON 文件中包含 {element_count} 个元素。")

JSON 文件中包含 302 个元素。


将文件夹中所有的json，merge到一起

In [3]:
import os
import json

def merge_json_files(input_folder, output_filename):
    # 创建一个空列表来存储所有的元素
    all_elements = []

    # 遍历文件夹中的所有JSON文件
    for filename in os.listdir(input_folder):
        if filename.endswith(".json"):
            filepath = os.path.join(input_folder, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                data = json.load(file)
                all_elements.extend(data)

    # 将所有元素写入到新的JSON文件中
    with open(output_filename, 'w', encoding='utf-8') as outfile:
        json.dump(all_elements, outfile, indent=4)

    print(f"已将文件夹中所有JSON文件中的元素合并到 '{output_filename}' 文件中。")

# 指定输入文件夹和输出文件名
input_folder = "/Users/qiangzhou/Library/Containers/com.tencent.xinWeChat/Data/Library/Application Support/com.tencent.xinWeChat/2.0b4.0.9/93c79210fac3454534da2b9ae0aab685/Message/MessageTemp/4fef58179a71273a0a5a8889e5523556/File/GAS_zhuoju/"
output_filename = "zhuoju_merged_output_GAS2SPAS.json"

# 调用函数来合并JSON文件
merge_json_files(input_folder, output_filename)


已将文件夹中所有JSON文件中的元素合并到 'zhuoju_merged_output_GAS2SPAS.json' 文件中。


In [2]:
import json

# 原始JSON文件名
input_json_filename = '/Users/qiangzhou/Library/CloudStorage/OneDrive-UniversityofPittsburgh/projects/Digital_mental_health/sentiment_dataset/negative_event_train.json'

# 读取原始JSON文件
with open(input_json_filename, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# 每个文件包含的元素数量
chunk_size = 50

# 分割数据并保存为多个JSON文件
for i in range(0, len(data), chunk_size):
    chunk_data = data[i:i+chunk_size]
    output_filename = f'output_chunk_{i//chunk_size + 1}.json'
    with open(output_filename, 'w', encoding='utf-8') as outfile:
        json.dump(chunk_data, outfile, indent=4)

print("Data has been successfully divided and saved into multiple JSON files.")


Data has been successfully divided and saved into multiple JSON files.


## 计算sentiment的分数


In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
from tqdm import tqdm
def get_sentiment_scores(text):
    MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    config = AutoConfig.from_pretrained(MODEL)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)

    def preprocess(text):
        new_text = []
        for t in text.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)

    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    results = []
    for i in range(scores.shape[0]):
        label = config.id2label[ranking[i]]
        score = scores[ranking[i]]
        results.append((label, np.round(float(score), 4)))
    sentiment_dict = {label: score for label, score in results}
    return sentiment_dict

def process_json_file(input_file, output_file):
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    for item in tqdm(data):
        post = item['Post']
        transferred_post = item['Transferred_Post']
        post_sentiment_scores = get_sentiment_scores(post)
        transferred_post_sentiment_scores = get_sentiment_scores(transferred_post)
        item['Post_Postive'] = post_sentiment_scores['positive']
        item['Post_Negative'] = post_sentiment_scores['negative']
        item['Post_Neutral'] = post_sentiment_scores['neutral']
        item['Transferred_Post_Postive'] = transferred_post_sentiment_scores['positive']
        item['Transferred_Post_Negative'] = transferred_post_sentiment_scores['negative']
        item['Transferred_Post_Neutral'] = transferred_post_sentiment_scores['neutral']

    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

input_file = "/home/qiang/projects/Digital_mental_health/Experiments/Sentiment_Scores/GAS2SPAS.json"
output_file = "/home/qiang/projects/Digital_mental_health/Experiments/Sentiment_Scores/GAS2SPAS_sentiment.json"
process_json_file(input_file, output_file)


NameError: name 'json' is not defined

In [ ]:
import json
import random

def select_random_elements(input_file, output_file, num_elements=10):
    # 读取输入 JSON 文件
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    # 随机选择指定数量的元素
    selected_elements = random.sample(data, min(num_elements, len(data)))
    
    # 写入输出 JSON 文件
    with open(output_file, 'w') as f:
        json.dump(selected_elements, f, indent=4)

# 输入和输出文件名
input_file = '/home/qiang/projects/Digital_mental_health/Dataset/Refined_dataset/0_origin_reid_filtered/GAS2SPAS.json'
output_file = 'output.json'

# 调用函数选择并保存随机元素
select_random_elements(input_file, output_file)
